In [1]:
import torch
from PIL import Image
import open_clip
import sys
sys.path.append("../util/")
from file_utils import pt_load

model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion400m_e32')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

/home/syoon/anaconda3/envs/paraclip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load pre-trained ParaCLIP weights
checkpoint = pt_load(
            "../data/ParaCLIP_weights/openclip_laion2b_vit-b-32/model.pt",
            map_location='cpu'
        )   
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [4]:
image = preprocess(Image.open("../data/dog.jpg")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[6.8896e-05, 9.9983e-01, 9.8660e-05]])
